In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

train_set = datasets.MNIST("./mnist/train", train=True, transform=transforms.ToTensor(), download=True)
test_set = datasets.MNIST("./mnist/test", train=False, transform=transforms.ToTensor(), download=True)

# generate indices: instead of the actual data we pass in integers instead
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_set)),
    train_set.targets,
    stratify=train_set.targets,
    train_size=0.8,
)

# generate subset based on indices
train_split = Subset(train_set, train_indices)
val_split = Subset(train_set, val_indices)


train_loader = DataLoader(train_split, batch_size=128)
val_loader = DataLoader(val_split, batch_size=128)
test_loader = DataLoader(test_set, batch_size=128)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()


        """
        Begining of Implement

        if num_layers is set to 1, we want the following structure:
            Linear(input_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, output_size)
        if num_layers is set to 2, we want the following structure:
            Linear(input_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, output_size)
        if num_layers is set to 3, we want the following structure:
            Linear(input_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, hidden_size)
            Activation function (no need to implement it here, we will take care of it in self.forward(...) )
            Linear(hidden_size, output_size)

        Here above are just examples, you can figure it out what it would be like when num_layers is set to 4
        """
        self.layers.append(nn.Linear(input_size, hidden_size))
        
        # last layer is output size 
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
        
        self.layers.append(nn.Linear(hidden_size, output_size))
        
        # print(self.layers) # verified to work
        

        """
        End of Implement
        """

    def forward(self, x):

        """
        Begining of of Implement

        1. the input x is of dimension (batch_size, 28, 28), you have to flatten it into a vector, i.e., of dimension (batch_size, 28*28)
        2. execute the forward computation of each layer, with activation function within (you can use torch.relu(x) to apply the act function)
        """
        vec = (x[0], x[1] * x[2])
        for layer in self.layers:
            layer.relu(vec)

        """
        End of Implement
        """
        return x

    # Training function
def train(model, criterion, optimizer, train_loader, val_loader, n_epochs):
    train_acc, val_acc = [], []
    for epoch in range(n_epochs):
        # Training
        model.train()
        correct, total = 0, 0
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            print('inputs:', inputs)
            """
            Begining of of Implement
            excecute the forward computation:
            1. get the model outputs -> variable 'outputs';
            2. compute loss -> variable 'loss'
            """

            ## your code here

            """
            End of Implement
            """





            """
            Begining of of Implement
            do the optimization step:
            1. clean the old/previous gradient;
            2. compute the current gradient (backward propagation)
            3. update the parameter
            """

            ## your code here

            """
            End of Implement
            """




            """
            Begining of of Implement
            get the predicted labels of the training data from variable 'outputs' -> variable 'predicted'

            """

            ## your code here

            """
            End of Implement
            """

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            if idx % 100 == 0:
                print(f"epoch: {epoch}, loss: {loss:.2f}, train_acc: {(correct/total):.2f}")

        train_acc.append(correct/total)

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                """
                Begining of of Implement
                1. get the model outputs -> variable 'outputs';
                2. get the predicted labels -> predicted
                """

                ## your code here

                """
                End of Implement
                """

                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        val_acc.append(correct/total)

    return train_acc, val_acc

# Testing function
def test(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            """
            Begining of of Implement
            1. get the model outputs -> variable 'outputs';
            2. get the predicted labels -> predicted
            """

            ## your code here

            """
            End of Implement
            """

            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct/total


input_size = 28*28
hidden_size = 32
output_size = 10
num_layers = [1, 2, 3, 4]
n_epochs = 5
lr = 0.01

# Training, Validation, and Testing
for num_layer in num_layers:
    model = MLP(input_size, hidden_size, num_layer, output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_acc, val_acc = train(model, criterion, optimizer, train_loader, val_loader, n_epochs)
    test_acc = test(model, test_loader)
    plt.plot(train_acc, label='Train Acc')
    plt.plot(val_acc, label='Val Acc')
    plt.legend(loc='lower right')
    plt.title(f'Num of Layers: {num_layer}, Test Acc: {test_acc}')
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.show()

inputs: tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0., 

NameError: name 'predicted' is not defined